In [2]:
import numpy as np
import pandas as pd

In [53]:
train = pd.read_csv('train_2016.csv')
prop = pd.read_csv('properties_2016.csv')

/Users/Leslie/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [91]:
prop['bed2bath'] = prop['bedroomcnt']/prop['bathroomcnt']
prop['bed2calbath'] = prop['bedroomcnt']/prop['calculatedbathnbr']
prop['areaperbedroom'] = prop['calculatedfinishedsquarefeet']/prop['bedroomcnt']
prop['livingarearate'] = prop['finishedsquarefeet12']/prop['finishedsquarefeet15']
prop['bed2fullbath'] = prop['bedroomcnt']/prop['fullbathcnt']
prop['areaperroom'] = prop['calculatedfinishedsquarefeet']/prop['roomcnt']
prop['houseage'] = 2016 - prop['yearbuilt']

In [92]:
for c, dtype in zip(prop.columns, prop.dtypes):	
    if dtype == np.float64:		
        prop[c] = prop[c].astype(np.float32)

df_train = train.merge(prop, how='left', on='parcelid')

x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)


(90811, 62) (90811,)


In [138]:
# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer

scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
def hyperopt_train_test(params):
    t = params['type']
    del params['type']
    if t == 'xgb':
        reg = xgb.XGBRegressor(**params)
    else:
        return 0, 0
    cv_score = cross_val_score(reg, x_train, y_train, scoring=scorer_mae, cv=5)
    score_mean = cv_score.mean()
    score_std = cv_score.std()
    return score_mean, score_std

space = hp.choice('classifier_type', [
    {
        'type': 'xgb',
        'learning_rate': hp.uniform('learning_rate', 0.02, 0.1),
        'min_child_weight': hp.choice('min_child_weight', range(30, 60)),
        'max_depth': hp.choice('max_depth', range(5, 10)),
        'subsample': hp.uniform('subsample', 0.2, 0.4),
        'n_estimators': hp.choice('n_estimators', range(150, 200)),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.4, 0.6),
        'reg_lambda': hp.uniform('reg_lambda', 1, 2),
        'reg_alpha': hp.uniform('reg_alpha', 1, 2)
}])

def f(params):
    global best, count
    count += 1
    result = hyperopt_train_test(params.copy())
    acc = result[0]
    std = result[1]
    print('iters:', count, ', acc:', acc, 'std:', std, '\nusing', params, '\n')
    return {'loss': -acc, 'status': STATUS_OK}

count = 0 
trials = Trials()
best = fmin(f, space, algo=tpe.suggest, max_evals=50, trials=trials)

## XGBoost

In [144]:
scorer_mae = make_scorer(mean_absolute_error)


In [155]:
for lr in np.linspace(0.06,0.064,5):
    reg = xgb.XGBRegressor(learning_rate=lr, n_estimators=100, max_depth=4, nthread=4, seed=1234)
    cv_score = cross_val_score(reg, x_train, y_train, scoring=scorer_mae, cv=3)
    score_mean = cv_score.mean()
    score_std = cv_score.std()
    if score_mean < 0.0696:
        print('learning_rate:{0}, Mean:{1:.8f}, STD:{2:.8f}'.format(lr, score_mean, score_std))

learning_rate:0.06, Mean:0.06947398, STD:0.00300854
learning_rate:0.061, Mean:0.06948449, STD:0.00301388
learning_rate:0.062, Mean:0.06947997, STD:0.00299754
learning_rate:0.063, Mean:0.06946019, STD:0.00302769
learning_rate:0.064, Mean:0.06943554, STD:0.00301250


In [157]:
# Score: 0.06943554
params = {'n_estimators': 100, 
          'learning_rate': 0.064, 
          'max_depth': 4,
          'nthread': 4,
          'gamma': 0.0001,
          'seed': 1234}
reg = xgb.XGBRegressor()
reg.set_params(**params)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.0001, learning_rate=0.064, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=100, nthread=4,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1234, silent=True, subsample=1)

In [158]:
reg.fit(x_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.0001, learning_rate=0.064, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=100, nthread=4,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1234, silent=True, subsample=1)

In [159]:
sample = pd.read_csv('sample_submission.csv')
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')

x_test = df_test[train_columns]

for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
# x_test = x_test.values.astype(np.float32, copy=False)

/Users/Leslie/anaconda3/lib/python3.5/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [160]:
p_test = reg.predict(x_test)

In [161]:
# p_test = 0.97*p_test + 0.03*0.011
sub = pd.read_csv('sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

sub.to_csv('result_xgb.csv', index=False, float_format='%.8f')

## Random Forest

In [162]:
from sklearn.ensemble import RandomForestRegressor
scorer_mae = make_scorer(mean_absolute_error)

In [188]:
x = pd.concat([x_train, x_test])

In [189]:
x = x.replace(np.inf, 0)

for var in x.columns[x_train.dtypes == 'object']:
    x[var] = x[var].fillna(x[var].mode())
    
for var in x.columns[x.dtypes != 'object']:
    x[var] = x[var].fillna(x[var].dropna().mean())

In [191]:
x_train = x[:90811]

(90811, 62)

In [195]:
print(x[90811:].shape)
print(x_test.shape)

(2985217, 62)
(2985217, 62)


In [ ]:
# n_estimators=200  --> 0.07474098
for est in [150,200,250,300]:
    reg = RandomForestRegressor(n_estimators=est)
    cv_score = cross_val_score(reg, x[:90811], y_train, scoring=scorer_mae, cv=3)
    score_mean = cv_score.mean()
    score_std = cv_score.std()
#     if score_mean < 0.0696:
    print('n_estimators:{0}, Mean:{1:.8f}, STD:{2:.8f}'.format(est, score_mean, score_std))

## LightGBM

In [56]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [93]:
xtrain, xvalid, ytrain, yvalid = train_test_split(x_train, y_train, 
                                                  test_size=0.3,
                                                  random_state=1234)

d_train = lgb.Dataset(xtrain, label=ytrain)
d_valid = lgb.Dataset(xvalid, label=yvalid)

In [113]:
params = {}
params['max_bin'] = 40
params['learning_rate'] = 0.0005  # 0.06890525
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l1'          # or 'mae'
params['feature_fraction'] = 0.8
params['sub_feature'] = 0.6      # feature_fraction 
params['bagging_freq'] = 50
params['num_leaves'] = 100       # num_leaf
params['min_data'] = 400         # min_data_in_leaf
params['min_hessian'] = 0.05 

# Score:0.06881544
# learning_rate: 0.0005
# feature_fraction: 0.8
# sub_feature:0.6
# bagging_freq:50
# num_leaves:100
# min_data:400

In [112]:
best = 0.06890525
watchlist = [d_valid]
for mh in [100,200,300,400]:
    params['min_hessian'] = md
    clf = lgb.train(params, d_train, 500, watchlist, verbose_eval=False)
    yvalid_pred = clf.predict(x_valid)
    mae = mean_absolute_error(y_pred=yvalid_pred, y_true=y_valid)
    if mae < best:
        print('Score:{0:.8f}, min_hessian:{1}'.format(mae, md))
#               print('Score:{0:.8f}, learning rate:{1}, max_bin:{2}, feature_fraction:{3}, bagging_freq:{4}, num_leaves:{5}'.format(mae, lr, m_bin, feat_frac, bag_freq, leaves))

Score:0.06884878, min_data:100
Score:0.06883632, min_data:200
Score:0.06882509, min_data:300
Score:0.06881544, min_data:400


In [114]:
watchlist = [d_valid]
clf = lgb.train(params, d_train, 500, watchlist, verbose_eval=False)

In [97]:
sample = pd.read_csv('sample_submission.csv')
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')

x_test = df_test[train_columns]

for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

/Users/Leslie/anaconda3/lib/python3.5/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [115]:
clf.reset_parameter({"num_threads":2})
p_test = clf.predict(x_test)

In [116]:
p_test.mean()

0.011075240581464639

In [118]:
p_test = 0.9*p_test + 0.1*0.011

sub = pd.read_csv('sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

sub.to_csv('result.csv', index=False, float_format='%.8f')

In [14]:
from sklearn.metrics import mean_absolute_error

In [11]:
y_valid_pred = clf.predict(x_valid)

In [29]:
y_art = 0.95*y_valid_pred + 0.05*0.0118

In [30]:
# 0.0718179
print('Score:', mean_absolute_error(y_pred=y_art, y_true=y_valid))

Score: 0.0718193790386


In [130]:
result = pd.read_csv('results/result_0.0646523.csv')

In [131]:
y_pred = result['201610']

In [132]:
result.columns

Index(['ParcelId', '201610', '201611', '201612', '201710', '201711', '201712'], dtype='object')

In [135]:
result['201611'] = y_pred*(1+0.022369)
result['201612'] = y_pred*(1+0.022369)**2
result['201710'] = y_pred*(1+0.022369)**6
result['201711'] = y_pred*(1+0.022369)**7
result['201712'] = y_pred*(1+0.022369)**8

In [136]:
result.to_csv('result_timeseries.csv', index=False, float_format='%.8f')